# Полный цикл подготовки TVR

Этот ноутбук собирает все шаги: от просмотра исходного TVR до формирования шаблона по маске и генерации итоговой таблицы.

In [2]:
import sys
from pathlib import Path

project_root = Path.cwd().resolve()
if not (project_root / 'src').exists():
    project_root = project_root.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
project_root

WindowsPath('C:/Users/user/Documents/piranha/constructor_TVR')

## Шаг 1. Загрузка TVR и выгрузка в Excel

1. Читаем исходный TVR-файл.
2. Сохраняем его в Excel, чтобы можно было визуально осмотреть все связи и stroka.
3. Смотрим первые строки для быстрой проверки.

**Что можно менять:** путь к `source_tvr_path` и имя сохранённого Excel.

In [3]:
from tvr_df import TVR_asis
import pandas as pd

source_tvr_path = Path(project_root) / 'Default.tvr2'  # при необходимости укажите другой исходник
excel_snapshot_path = Path(project_root) / 'docs' / 'default_snapshot.xlsx'

tvr_full = TVR_asis(source_tvr_path)
tvr_full.to_excel(excel_snapshot_path, index=False)

print(f'Excel со всеми строками сохранён: {excel_snapshot_path}')
tvr_full.head(10)

Excel со всеми строками сохранён: C:\Users\user\Documents\piranha\constructor_TVR\docs\default_snapshot.xlsx


,stroka,Start,Kill all,Out only,InL1,InL2,OutL1,OutL2,Pos,Limit,...,T.In,T.Out,NotSet,FrId,MoveN,secIn,secOut,XN,MP,xx
0,1,NaN,NaN,NaN,█████,█████,█████,█████,█████,NaN,...,█████,█████,█████,█████,█████,█████,█████,█████,█████,█████
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28,NaN,NaN,NaN,█████,█████,█████,█████,█████,NaN,...,█████,█████,█████,█████,█████,█████,█████,█████,█████,█████
3,29,NaN,NaN,NaN,31-170,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30,NaN,NaN,NaN,31-32,█████,█████,█████,█████,NaN,...,█████,█████,█████,█████,█████,█████,█████,█████,█████,█████
5,31,True,NaN,NaN,33,NaN,33,NaN,NaN,NaN,...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,NaN,3/1,3/3,NaN,NaN,0.005,NaN,NaN
6,32,True,NaN,NaN,33,NaN,33,NaN,NaN,NaN,...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,NaN,3/1,3/3,NaN,NaN,0.005,NaN,NaN
7,33,NaN,NaN,NaN,34,NaN,34,NaN,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,3/1,NaN,NaN,NaN,NaN,NaN,NaN
8,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,40,NaN,NaN,NaN,41-42,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Шаг 2. Выбор базовых строк для парсинга

Передаём парсеру stroka базовых блоков (обычно long/short).
1. Для ориентира можно отфильтровать строки по `Mode` и посмотреть кандидатов.
2. После этого явно задаём `selected_strokas` — именно этот список пойдёт дальше.

**Что можно менять:** значение `target_mode` и финальный список `selected_strokas`.

In [4]:
from IPython.display import display

target_mode = 1  # 1 — long, -1 — short
mode_candidates = (
    tvr_full.loc[pd.to_numeric(tvr_full['Mode'], errors='coerce') == target_mode, ['stroka', 'Mode', 'Pos']]
    .reset_index(drop=True)
)

print('Кандидаты по Mode ==', target_mode)
display(mode_candidates.head(10))

Кандидаты по Mode == 1


,stroka,Mode,Pos
0,31,1,NaN
1,41,1,NaN
2,51,1,NaN
3,61,1,NaN
4,71,1,NaN
5,81,1,NaN
6,91,1,NaN
7,101,1,NaN
8,111,1,NaN
9,121,1,NaN


In [5]:
# Пример: вручную выбраны stroka для разбора (long и short)
selected_strokas = [5462, 5466]
selected_strokas

[5462, 5466]

## Шаг 3. Переориентация выбранного шаблона

Загружаем структуру только по нужным stroka, выводим её в текстовом виде, вручную задаём новые смещения и компилируем результат.

**Что можно менять:**
- `target_base_stroka` — абсолютный номер, куда ставим базовую строку.
- содержимое `edited_text` — новые offsets для нужных узлов.

In [6]:
from src.tvr_service.generator import build_layout_from_source
from src.tvr_service.generator.layout import LayoutEdits, compile_layout

layout = build_layout_from_source(str(source_tvr_path), selected_strokas)
print(f'Основная база (до переориентации): {layout.primary_base_stroka}')

Основная база (до переориентации): 5462


### Структура в текстовом виде

Скопируйте блок ниже и правьте offsets вручную (значение после двоеточия).

In [7]:
seen = {}
lines = []

for entry in layout.entries:
    base_alias = entry.label.replace(" ", "_")
    counter = seen.get(base_alias, 0)
    seen[base_alias] = counter + 1

    alias = base_alias if counter == 0 else f"{base_alias}__{counter+1}"

    lines.append(f"{alias}: {entry.relative_offset}")
layout_text = '\n'.join(lines)
print(layout_text)


# lines = []
# for entry in layout.entries:
#     label = entry.label.replace(' ', '_')
#     lines.append(f"{label}: {entry.relative_offset}")
# layout_text = '\n'.join(lines)
# print(layout_text)

base_long: 0
filter_1_long_&_short: 8
filter_2_long_&_short: 12
filter_3_long_&_short: 13
filter_4_long_&_short: 14
filter_5_long_&_short: 16
filter_6_long_&_short: 17
filter_7_long_&_short: 18
filter_3_long_&_short__2: 15
base_short: 4


In [8]:
selected_strokas = [5462, 5466]

### Редактируем смещения вручную

Ниже пример со смещениями: после второго двоеточия указано новое значение.
При необходимости замените на свои цифры.

In [9]:
edited_text = '''base_long: 0
filter_1_long_&_short: 8:3
filter_2_long_&_short: 12:4
filter_3_long_&_short: 13:5
filter_4_long_&_short: 14:6
filter_5_long_&_short: 16:8
filter_6_long_&_short: 17:9
filter_7_long_&_short: 18:10
filter_3_long_&_short__2: 15:7
base_short: 4:1
'''

target_base_stroka = 1000  # куда переносим базовую строку

def parse_simple_overrides(layout, text):
    overrides = {}
    for line in text.strip().splitlines():
        if ':' not in line:
            continue
        name, *rest = [chunk.strip() for chunk in line.split(':')]
        if len(rest) < 2 or rest[1] == '':
            continue
        new_offset = int(rest[1])
        for entry in layout.entries:
            if entry.label.replace(' ', '_') == name:
                overrides[entry.original_stroka] = new_offset
                break
    return LayoutEdits(relative_overrides=overrides)

edits = parse_simple_overrides(layout, edited_text)
compiled = compile_layout(layout, edits, base_assignment={layout.primary_base_stroka: target_base_stroka})
compiled

CompiledLayout(base_assignment={5462: 1000, 5466: 1001}, stroka_overrides={5470: 1003, 5474: 1004, 5475: 1005, 5476: 1006, 5478: 1008, 5479: 1009, 5480: 1010})

## Шаг 4. Заготовка маски в Excel

На основе пересобранного шаблона формируем таблицу с `row_alias` и столбцами TVR.
Оператор отмечает нужные ячейки значением `1` и сохраняет файл как `mask.xlsx`.

Файл включает:
- блок шаблона сверху;
- три пустые строки-разделителя;
- пример строк из исходного робота (для ориентира значения).

**Что можно менять:** список дополнительных колонок или stroka, которые нужно добавить в заготовку.

In [10]:
mask_template_records = []
seen = {}

for entry in layout.entries:
    base_alias = entry.label.replace(" ", "_")
    counter = seen.get(base_alias, 0)
    seen[base_alias] = counter + 1
    alias = base_alias if counter == 0 else f"{base_alias}__{counter+1}"
    absolute = compiled.stroka_overrides.get(entry.original_stroka)
    if absolute is None:
        absolute = compiled.base_assignment.get(entry.original_stroka)
    if absolute is None:
        absolute = target_base_stroka + entry.relative_offset


    # absolute = ...
    mask_template_records.append({"row_alias": alias, "stroka": absolute})



# mask_template_records = []
# for entry in layout.entries:
#     alias = entry.label.replace(' ', '_')
#     absolute = compiled.stroka_overrides.get(entry.original_stroka)
#     if absolute is None:
#         absolute = compiled.base_assignment.get(entry.original_stroka)
#     if absolute is None:
#         absolute = target_base_stroka + entry.relative_offset
#     mask_template_records.append({'row_alias': alias, 'stroka': absolute})

# Полный перечень колонок TVR
tvr_columns = list(tvr_full.columns)
mask_columns = ['row_alias', *tvr_columns]

mask_template_df = pd.DataFrame(mask_template_records)
mask_template_df = mask_template_df.reindex(columns=mask_columns, fill_value=pd.NA)

# Три пустые строки-разделителя
separator_df = pd.DataFrame([{col: pd.NA for col in mask_columns} for _ in range(3)])

# Пример строк из исходного робота для наглядности
sample_strokas = sorted([entry.original_stroka for entry in layout.entries])
# sample_strokas = [5462, 5466]  # пример, можно заменить
sample_df = tvr_full[tvr_full['stroka'].isin(sample_strokas)].copy()
sample_df.insert(0, 'row_alias', [f'sample_{int(st)}' for st in sample_df['stroka']])
sample_df = sample_df.reindex(columns=mask_columns)

mask_output_df = pd.concat([mask_template_df, separator_df, sample_df], ignore_index=True, sort=False)
mask_template_path = Path(project_root) / 'docs' / 'mask_template.xlsx'
mask_output_df.to_excel(mask_template_path, index=False)

print(f'Шаблон маски сохранён: {mask_template_path}')
mask_output_df.head(15)

Шаблон маски сохранён: C:\Users\user\Documents\piranha\constructor_TVR\docs\mask_template.xlsx


,row_alias,stroka,Start,Kill all,Out only,InL1,InL2,OutL1,OutL2,Pos,...,T.In,T.Out,NotSet,FrId,MoveN,secIn,secOut,XN,MP,xx
0,base_long,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,filter_1_long_&_short,1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,filter_2_long_&_short,1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,filter_3_long_&_short,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,filter_4_long_&_short,1006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,filter_5_long_&_short,1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,filter_6_long_&_short,1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,filter_7_long_&_short,1010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,filter_3_long_&_short__2,1015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,base_short,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mask_path = Path(project_root) / 'mask.xlsx'
mask_df = pd.read_excel(mask_path)
print('Текущая маска:')
mask_df

from src.tvr_service.templates import build_template_from_mask_file

mask_result = build_template_from_mask_file(
    mask_path,
    row_alias_column='row_alias',
    marker=1,
)

print('Колонки для конфигурации:')
mask_result.override_columns

In [11]:
from src.tvr_service.templates import build_template_from_mask_file

mask_path = Path(project_root) / "mask.xlsx"
mask_df = pd.read_excel(mask_path)
print("Текущая маска:")
mask_df



Текущая маска:


,row_alias,stroka,Sec 0,V 0,Sec 1,V 1,W 1,SM
0,base_long,1000,1,1.0,1.0,NaN,NaN,1.0
1,base_short,1001,1,1.0,1.0,NaN,NaN,1.0
2,filter_1_long_&_short,1003,1,NaN,1.0,NaN,NaN,NaN
3,filter_2_long_&_short,1004,1,NaN,NaN,NaN,1.0,NaN
4,filter_3_long_&_short,1005,1,NaN,NaN,NaN,1.0,NaN
5,filter_4_long_&_short,1006,1,NaN,1.0,NaN,NaN,NaN
6,filter_5_long_&_short,1008,1,NaN,1.0,NaN,NaN,NaN
7,filter_6_long_&_short,1009,1,NaN,NaN,NaN,1.0,NaN
8,filter_7_long_&_short,1010,1,NaN,NaN,NaN,1.0,NaN
9,filter_3_long_&_short__2,1015,1,NaN,1.0,NaN,NaN,NaN


In [12]:
mask_unique = mask_df['row_alias'].dropna().duplicated()
mask_df[mask_unique]

,row_alias,stroka,Sec 0,V 0,Sec 1,V 1,W 1,SM


In [13]:


mask_result = build_template_from_mask_file(
    mask_path,
    row_alias_column='stroka',
    marker=1,
)

print('Колонки для конфигурации:')
mask_result.override_columns

ValueError: Row aliases must be unique within a template

## Шаг 6. Подготовка конфигурации и генерация TVR

Используем список колонок из маски, заполняем конфигурацию и строим итоговый блок.

**Что можно менять:** значения в `default_overrides`, `strategy_id`, `start`, а также параметры генератора.

In [1]:
from src.tvr_service.generator import StrategyGenerator

default_overrides = {}
for name in mask_result.override_columns:
    if name.endswith('Sec_0'):
        default_overrides[name] = 'SEC_DEMO'
    elif any(name.endswith(suffix) for suffix in ('V_0', 'C', 'N', 'P')):
        default_overrides[name] = 1
    elif name.endswith('W_1'):
        default_overrides[name] = 100
    elif name.endswith('SM'):
        default_overrides[name] = 5
    else:
        default_overrides[name] = 0

config_table = pd.DataFrame([
    {
        'strategy_id': 'demo_strategy',
        'start': 500,
        **default_overrides,
    }
])
config_table

ModuleNotFoundError: No module named 'src'

In [ ]:
generator = StrategyGenerator(
    mask_result.template,
    start_column='start',
    strategy_column='strategy_id',
    sec_column=None,
)

tvr_result = generator.generate(config_table, blank_rows_between=1)
tvr_result